# 02 - Data cleaning 

## Overview
1. **Load data**: Import dataset from the previous stage.
2. **Convert dtypes:** Apply selected column type conversions based on decisions from Stage 01.
3. **Standarize columns names:** Rename all columns to `snake_case` and remove non-breaking spaces.  
4. **Column cleanup:** Drop redundant `Row ID` and `Country` columns.
5. **Save changes:** export cleaned version of the dataset to:  
   `data/interim/02_silver_clean.csv` (Format changed to `.parquet`)

**Goal:** Fix formatting so every column is usable.

### Load data
> Import Stage 01 CSV file 

In [4]:
# Import Pandas
import pandas as pd

# Load bronze stage data
file_path = '../data/interim/01_bronze_intake.csv'
df = pd.read_csv(file_path)

# Check results
print('Data loaded successfully.')
df.head()

Data loaded successfully.


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


### Convert dtypes
> Change dtypes according finding from the previous stage.

Take column idexes manualy from previous notebook.

In [7]:
# Copy comlums indexes from 01_data_intake.ipynb
str_cols = [1, 5, 6, 9, 11, 13, 16]
cat_cols = [4, 7, 8, 10, 12, 14, 15]
dt_cols = [2, 3]

# Get column names by index
str_cols = df.columns[str_cols]
cat_cols = df.columns[cat_cols]
dt_cols = df.columns[dt_cols]

# Convert dtypes

# Convert to 'string[pyarrow]' instead of 'string[python]' better speed and smaller memory footprint
df[str_cols]= df[str_cols].astype('string[pyarrow]') 

df[cat_cols]= df[cat_cols].astype('category')

for col in dt_cols:
    df[col] = pd.to_datetime(df[col], dayfirst=False, yearfirst=True)

# Check reults
df.dtypes

Row ID                     int64
Order ID         string[pyarrow]
Order Date        datetime64[ns]
Ship Date         datetime64[ns]
Ship Mode               category
Customer ID      string[pyarrow]
Customer Name    string[pyarrow]
Segment                 category
Country                 category
City             string[pyarrow]
State                   category
Postal Code      string[pyarrow]
Region                  category
Product ID       string[pyarrow]
Category                category
Sub-Category            category
Product Name     string[pyarrow]
Sales                    float64
Quantity                   int64
Discount                 float64
Profit                   float64
dtype: object

### Standarize columns names
> Rename all columns to `snake_case` and remove non-breaking spaces for consistency and readability.

In [9]:
df.columns = (
    df.columns
      .str.strip()                 # remove extra spaces
      .str.replace(r'\s+', '_', regex=True)  # spaces → underscores
      .str.replace('-', '_', regex=False)     # dashes → underscores
      .str.lower()                 # lowercase
)

# Check results
df.columns 

Index(['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode',
       'customer_id', 'customer_name', 'segment', 'country', 'city', 'state',
       'postal_code', 'region', 'product_id', 'category', 'sub_category',
       'product_name', 'sales', 'quantity', 'discount', 'profit'],
      dtype='object')

### Column cleanup 
> rename, remove empty, duplicated or constant columns.

In the previous stage, we identified `row_id` and `country` as non-informative features. Since they provide no analytical value, they will be removed.

In [12]:
# Drop columns
df = df.drop(columns = ['row_id','country'])

# Check results
print('Column removed succesfully')
df.shape
df.columns

Column removed succesfully


Index(['order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_id',
       'customer_name', 'segment', 'city', 'state', 'postal_code', 'region',
       'product_id', 'category', 'sub_category', 'product_name', 'sales',
       'quantity', 'discount', 'profit'],
      dtype='object')

### Save changes
> Scince cleaning stage is complete, export cleaned version of the dataset.

Change export file format to Parquet in order to preserve dtypes.

In [15]:
df.to_parquet('../data/interim/02_silver_cleaned.parquet')
print('Copy is saved')

Copy is saved


A summary of this stage is documented separately in `/reports/02_data_cleaning.md`.